In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top5leagues/top5/la_liga/2019-2020.csv
/kaggle/input/top5leagues/top5/la_liga/2018-2019.csv
/kaggle/input/top5leagues/top5/la_liga/2017-2018.csv
/kaggle/input/top5leagues/top5/la_liga/2016-2017.csv
/kaggle/input/top5leagues/top5/la_liga/2020-2021.csv
/kaggle/input/top5leagues/top5/la_liga/2021-2022.csv
/kaggle/input/top5leagues/top5/la_liga/2014-2015.csv
/kaggle/input/top5leagues/top5/la_liga/2015-2016.csv
/kaggle/input/top5leagues/top5/serie_a/2019-2020.csv
/kaggle/input/top5leagues/top5/serie_a/2018-2019.csv
/kaggle/input/top5leagues/top5/serie_a/2017-2018.csv
/kaggle/input/top5leagues/top5/serie_a/2016-2017.csv
/kaggle/input/top5leagues/top5/serie_a/2020-2021.csv
/kaggle/input/top5leagues/top5/serie_a/2021-2022.csv
/kaggle/input/top5leagues/top5/serie_a/2014-2015.csv
/kaggle/input/top5leagues/top5/serie_a/2015-2016.csv
/kaggle/input/top5leagues/top5/epl/2019-2020.csv
/kaggle/input/top5leagues/top5/epl/2018-2019.csv
/kaggle/input/top5leagues/top5/epl/2017-2018.csv
/kagg

In [3]:
import plotly.express as px
import plotly.graph_objs as go

table = ["2021-2022", "2020-2021", "2019-2020", "2018-2019", "2017-2018", "2016-2017"]

ll = {}
for t in table:
    ll[t] = pd.read_csv(f"/kaggle/input/top5leagues/top5/la_liga/{t}.csv")

total_tables = []
for t in table:
    total_table = (
        ll[t].groupby("HomeTeam")["FTHG"].sum()
        .add(ll[t].groupby("AwayTeam")["FTAG"].sum(), fill_value=0)
        .reset_index(name="TotalGoals")
        .rename(columns={"HomeTeam":"Team"})
    )
    total_table.insert(0, "Season", f"{t}")
    total_tables.append(total_table)

all_seasons = pd.concat(total_tables, ignore_index=True)

fig = go.Figure()

for i, season in enumerate(table):
    visible = [False] * len(table)
    visible[i] = True

    season_data = all_seasons[all_seasons['Season'] == season]
    sorted_data = season_data.sort_values(by='TotalGoals', ascending=False)
    fig.add_trace(go.Scatter(
        x=sorted_data['Team'],
        y=sorted_data['TotalGoals'],
        text=sorted_data['TotalGoals'],
        visible=True if i == 0 else False,
        name=season
    ))

updatemenus = [dict(
    active=0,
    buttons=list([
        dict(label=season, method="update", args=[{"visible": visible},
                                                 {"title": "Goals Scored in La Liga in " + season}])
        for season, visible in zip(table, [[i == j for j in range(len(table))] for i in range(len(table))])
    ]))
]

fig.update_layout(updatemenus=updatemenus, xaxis_title="Team", yaxis_title="Total Goals", title="Goals Scored in La Liga")

fig.show()



In [28]:
br = all_seasons[(all_seasons["Team"] == "Barcelona") | (all_seasons["Team"] == "Real Madrid")]

fig = px.line(br, x='Season', y='TotalGoals', color='Team')
fig.update_layout(title_text="Barcelona vs Real Madrid (Total goals by season)", xaxis_title="Season", yaxis_title="Total Goals")
fig.show()



In [52]:
table_bl = ["2016-2017", "2017-2018","2018-2019","2019-2020","2020-2021","2021-2022"]

bl = {}
for t in table_bl:
    bl[t] = pd.read_csv(f"/kaggle/input/top5leagues/top5/bundesliga/{t}.csv")
    
total_tables_bl = []
for t in table:
    total_table_bl = (
        bl[t].groupby("HomeTeam")["FTHG"].sum()
        .add(ll[t].groupby("AwayTeam")["FTAG"].sum(), fill_value=0)
        .reset_index(name="TotalGoals")
        .rename(columns={"HomeTeam":"Team"})
    )
    total_table_bl.insert(0, "Season", f"{t}")
    total_tables_bl.append(total_table_bl)

all_seasons_bl = pd.concat(total_tables_bl, ignore_index=True)
bl = all_seasons_bl.sort_values("Season", ascending=True)

fig = px.line(bl, x="Season", y="TotalGoals", color="Team", line_group="Team", hover_name="Team",
        line_shape="spline", render_mode="svg")
fig.show()

        Season               Team  TotalGoals
107  2016-2017          Wolfsburg          34
90   2016-2017           Augsburg          35
91   2016-2017   Bayer Leverkusen          53
92   2016-2017      Bayern Munich          89
93   2016-2017  Borussia Dortmund          72
..         ...                ...         ...
15   2021-2022       Union Berlin          50
16   2021-2022      VfB Stuttgart          41
17   2021-2022          Wolfsburg          43
9    2021-2022           Freiburg          58
0    2021-2022  Arminia Bielefeld          27

[108 rows x 3 columns]


In [96]:
ll2 = pd.read_csv('/kaggle/input/top5leagues/top5/la_liga/2021-2022.csv').loc[:,["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
ll3 = ll2.groupby("HomeTeam")["FTHG"].sum().add(ll2.groupby("AwayTeam")["FTAG"].sum()).reset_index(name="TotalGoals").rename(columns={"HomeTeam":"Team"}).sort_values("TotalGoals", ascending=False).head(1)


sa = pd.read_csv('/kaggle/input/top5leagues/top5/serie_a/2021-2022.csv').loc[:,["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
sa_max = sa.groupby("HomeTeam")["FTHG"].sum().add(sa.groupby("AwayTeam")["FTAG"].sum()).reset_index(name="TotalGoals").rename(columns={"HomeTeam":"Team"}).sort_values("TotalGoals", ascending=False).head(1)

epl = pd.read_csv('/kaggle/input/top5leagues/top5/epl/2021-2022.csv').loc[:,["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
epl_max = epl.groupby("HomeTeam")["FTHG"].sum().add(epl.groupby("AwayTeam")["FTAG"].sum()).reset_index(name="TotalGoals").rename(columns={"HomeTeam":"Team"}).sort_values("TotalGoals", ascending=False).head(1)


l1 = pd.read_csv('/kaggle/input/top5leagues/top5/ligue_1/2021-2022.csv').loc[:,["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
l1_max = l1.groupby("HomeTeam")["FTHG"].sum().add(l1.groupby("AwayTeam")["FTAG"].sum()).reset_index(name="TotalGoals").rename(columns={"HomeTeam":"Team"}).sort_values("TotalGoals", ascending=False).head(1)


bl = pd.read_csv('/kaggle/input/top5leagues/top5/bundesliga/2021-2022.csv').loc[:,["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
bl_max = bl.groupby("HomeTeam")["FTHG"].sum().add(bl.groupby("AwayTeam")["FTAG"].sum()).reset_index(name="TotalGoals").rename(columns={"HomeTeam":"Team"}).sort_values("TotalGoals", ascending=False).head(1)


concatenated_df = pd.concat([ll3, sa_max, epl_max, l1_max, bl_max], axis=0)

fig = px.bar(concatenated_df, x='Team', y='TotalGoals')
fig = fig.update_layout(title='Most goals in the league in the 2021-2022 season')
fig = fig.update_traces(marker_color='blue', marker_line_color='rgb(8,48,107)')
fig = fig.update_xaxes(categoryorder='total descending')
fig.show()
